In [42]:
import os
import sys
project_dir = os.path.join(os.pardir, os.pardir)
sys.path.append(project_dir)

import pandas as pd
import numpy as np
import pulp
import time
from IPython.display import display
from shapely.geometry import Polygon

In [43]:
#Data reading and cleaning
df_sectors = pd.read_csv(project_dir + "/data/external/od/setores_censitarios_CSV.csv",
                         encoding="latin-1")
df_sectors.WKT = df_sectors.WKT.str.replace("POLYGON", "")
df_sectors.WKT = df_sectors.WKT.str.replace("\(", "")
df_sectors.WKT = df_sectors.WKT.str.replace("\)", "")
df_sectors.WKT = df_sectors.WKT.str.strip()

#Extract coordinates from string
df_sectors["coords"] = df_sectors.WKT.str.split(",")
df_sectors.coords = df_sectors.coords.apply(lambda x: [tuple(map(float, i.split(" "))) for i in x])

#Create shapely Polygon
df_sectors["polygon"] = df_sectors.coords.apply(lambda x: Polygon(x))

#Get polygon area
df_sectors["area"] = df_sectors.polygon.apply(lambda x: x.area)

#Check data
print(len(df_sectors), "rows")
print(df_sectors.objectid.nunique(), "objectids")

#Drop unused columns and index with cd_geocodi
df_sectors = df_sectors[["cd_geocodi", "tipo", "nm_bairro", "polygon", "area"]]
df_sectors.set_index("cd_geocodi", inplace=True, verify_integrity=True)

#Remove incorrect sectors
incorrect_sectors = pd.read_csv(project_dir + "/data/external/od/sector_errors.csv",
                                encoding="latin-1",
                                sep=";")

incorrect_sectors_list = incorrect_sectors.cd_geocodi.tolist()
df_sectors.drop(incorrect_sectors_list, inplace=True)

#Check data
print(len(df_sectors), "rows")

df_sectors.head()

730 rows
730 objectids
715 rows


,tipo,nm_bairro,polygon,area
cd_geocodi,,,,
420910205000157,URBANO,Nova Brasília,"POLYGON ((712952.1411 7085390.3829, 713025.628...",1.869259e+05
420910205000158,URBANO,Nova Brasília,"POLYGON ((712792.4804 7085600.7155, 712956.814...",3.038262e+05
420910205000159,URBANO,Nova Brasília,"POLYGON ((712088.8612 7085780.435, 712102.1324...",5.428793e+05
420910205000160,URBANO,Nova Brasília,"POLYGON ((712445.1203 7085003.5311, 712818.006...",2.844532e+06
420910205000162,URBANO,São Marcos,"POLYGON ((710085.5298 7089374.0183, 710384.236...",2.165143e+06


 Number of Equipments and Census Zones:

In [44]:
sector_list = df_sectors.index.unique().tolist()
max_setores = len(sector_list)
print("Initial number of sectors: ",max_setores)

df_equip = pd.read_csv(project_dir + "/data/interim/od/avg_per_quarter_815.csv")
df_equip.drop(df_equip.columns[0], axis=1, inplace=True)
equip_list = df_equip["Equipamento"].unique().tolist()
max_radares = len(equip_list)
print("Initial number of radars: ", max_radares)
display(df_equip.head())


Initial number of sectors:  715
Initial number of radars:  95


,Endereco,Sentido,Equipamento,Latitude,Longitude,Horario,Numero de Faixas,Corredor,Total,total_por_faixa,weighted_avg_speed
0,Av.Dr. Albano Schultz x Rua Princesa Izabel,Norte/Sul,FS569JOI,-26.2998,-48.8424,08:00 as 08:14,2.0,1.0,470.5,235.25,27.228464
1,Rua Iririu. 1070,N/S,FS579JOI,-26.27599,-48.83419,08:00 as 08:14,2.0,0.0,426.5,213.25,43.692534
2,Rua Blumenau nº1580,Norte/Sul,FS633JOI,-26.28678,-48.84984,08:00 as 08:14,2.0,1.0,409.5,204.75,40.970551
3,Rua Marquês de Olinda. 2841,N/S,FS653JOI,-26.28913,-48.86434,08:00 as 08:14,1.0,0.0,193.0,193.00,47.129987
4,Rua Iririu 246,Norte/Sul,FS599JOI,-26.28206,-48.83903,08:00 as 08:14,2.0,1.0,381.5,190.75,30.525773


Note that several equipment have flow in two directions:

In [45]:
direction_per_equip = df_equip.groupby("Equipamento").agg({"Sentido": "count"})
direction_per_equip.sort_values("Sentido", ascending=False).head()

,Sentido
Equipamento,
FS551JOI,2
FS580JOI,2
FS583JOI,2
FS585JOI,2
FS589JOI,2


We will use the MAX of the flow in both directions as proxy of flow through that equipment.

In [46]:
flow_per_equip = df_equip.groupby("Equipamento").agg({"Total": "max"})
flow_per_equip["proporcao_fluxo"] = flow_per_equip["Total"] / flow_per_equip["Total"].sum()
flow_per_equip["ideal_num_sectors"] = flow_per_equip["proporcao_fluxo"]*max_setores
flow_per_equip["ideal_num_sectors"] = flow_per_equip["ideal_num_sectors"].round()
diff = round(flow_per_equip["ideal_num_sectors"].sum() - max_setores)
flow_per_equip.iloc[0:abs(diff), flow_per_equip.columns.get_loc("ideal_num_sectors")] -= np.sign(diff)
flow_per_equip["ideal_num_sectors"] = flow_per_equip["ideal_num_sectors"].astype(int)
print("Number of sectors after allocation: ", flow_per_equip["ideal_num_sectors"].sum())
print("Checking number of equipment: ", len(flow_per_equip))
flow_per_equip.head()

Number of sectors after allocation:  715
Checking number of equipment:  95


,Total,proporcao_fluxo,ideal_num_sectors
Equipamento,,,
FS551JOI,56.0,0.003160,3
FS552JOI,92.0,0.005192,4
FS555JOI,103.0,0.005813,4
FS556JOI,154.5,0.008719,6
FS557JOI,95.0,0.005361,4


Get Distance Matrix exported from QGis:

In [47]:
df_distance_matrix = pd.read_csv(project_dir + "/data/external/od/Matriz_distancias.csv", index_col=0)
df_distance_matrix.drop(incorrect_sectors_list, inplace=True)
all_cols = df_distance_matrix.columns.tolist()
valid_equip = flow_per_equip.index.tolist()
valid_cols = [col for col in all_cols if col in valid_equip]
df_distance_matrix = df_distance_matrix[valid_cols]
df_distance_matrix.head()

,FS563JOI,FS607JOI,FS603JOI,FS573JOI,FS564JOI,FS650JOI,FS592JOI,FS586JOI,FS572JOI,FS610JOI,...,FS587JOI,FS577JOI,FS643JOI,FS614JOI,FS583JOI,FS561JOI,FS551JOI,FS652JOI,FS552JOI,FS580JOI
ID,,,,,,,,,,,,,,,,,,,,,
420910205000157,1555.348538,2078.813000,2214.474220,2230.905122,2324.336417,2513.556114,2847.362218,2935.045195,2951.610919,2973.540954,...,8655.603021,8721.982207,8774.110669,9183.149427,9315.959129,9765.764834,10072.837073,11439.761384,12696.624012,14434.885892
420910205000158,2161.843243,2577.932317,2787.289889,2371.455982,2425.207904,2537.106058,3074.921395,3280.455078,3461.909925,3505.949563,...,8877.796742,9058.381584,8822.886836,9412.738303,9587.444872,9971.873880,10394.178557,11605.777350,12441.766897,14524.548969
420910205000159,2772.289767,3207.825781,3414.882190,2871.042824,2899.679359,2956.794049,3606.756216,3865.104950,4092.075329,4137.592986,...,9371.518238,9619.627589,9189.383825,9910.101005,10110.861648,10452.514539,10945.535770,12055.346090,12505.437472,14911.613876
420910205000160,2350.004824,3251.342774,3092.235856,3838.506835,3945.407447,4151.044734,4384.608323,4337.916067,3996.385785,3937.698963,...,10123.059963,10018.738985,10383.246051,10638.743889,10714.486005,11246.449886,11386.322439,12957.404603,14283.811510,16011.897783
420910205000162,4754.545623,4377.473509,4935.131679,2837.352035,2683.377207,2365.534803,3330.050424,3982.633850,4990.237224,5164.634254,...,7562.690863,8427.181640,6588.873393,8095.960754,8505.511064,8480.149934,9565.184785,9804.474507,9073.606012,12250.738718


Build Adjacency Matrix:

In [48]:
start = time.time()
adjacency_matrix = pd.DataFrame(0, index=df_sectors.index, columns=df_sectors.index)
def build_adjacency_matrix():
    for index1,_ in df_sectors.iterrows():
        for index2,_ in df_sectors.iterrows():
            if index1 == index2:
                intersects = 1
            else:
                try:
                    intersects = int(df_sectors.loc[index1, "polygon"].intersects(df_sectors.loc[index2, "polygon"]))
                except:
                    intersects = 1 #errors are probably being caused by wrong Linestrings in adjacent Polygons                                        

            adjacency_matrix.loc[index1, index2] = intersects
            
            return adjacency_matrix
        
#adjacency_matrix = build_adjacency_matrix()
adjacency_matrix = pd.read_excel(project_dir + "/data/interim/od/adjacency_matrix.xlsx", index_col=0)
end = time.time()
elapsed_time = str(int(end-start))
print("Matrix created in " + elapsed_time + " seconds.")

adjacency_matrix.head()

Matrix created in 11 seconds.


,420910205000157,420910205000158,420910205000159,420910205000160,420910205000162,420910205000163,420910205000164,420910205000007,420910205000165,420910205000166,...,420910210000011,420910210000012,420910210000013,420910210000014,420910210000015,420910210000016,420910205000734,420910210000029,420910210000030,420910210000031
cd_geocodi,,,,,,,,,,,,,,,,,,,,,
420910205000157,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420910205000158,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420910205000159,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420910205000160,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420910205000162,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Clustering algorithm - sweeping sectors

In [49]:
temp_dist_matrix = df_distance_matrix.copy()
clustering_df_s = flow_per_equip.copy()
clustering_df_s["avg_distance"] = 0
clustering_df_s["sector_list"] = [[] for _ in range(len(flow_per_equip))]

#Initialization
for equip, data in clustering_df_s.iterrows():
    closest_sector = temp_dist_matrix.loc[:,equip].idxmin()
    clustering_df_s.loc[equip, "avg_distance"] = temp_dist_matrix.loc[closest_sector, equip]
    temp_list = clustering_df_s.loc[equip, "sector_list"]
    temp_list.append(closest_sector)
    clustering_df_s.set_value(equip, "sector_list", temp_list)
    temp_dist_matrix.drop(closest_sector, inplace=True)
    
#Allocation
while len(temp_dist_matrix) > 0:
    for sector, distance in temp_dist_matrix.iterrows():
        equip_to_append = None
        sorted_dist = temp_dist_matrix.loc[sector].sort_values()
        for equip, dist in sorted_dist.iteritems():
            sector_list = clustering_df_s.loc[equip, "sector_list"]
            adj = False
            for cluster_sector in sector_list:
                if adjacency_matrix.loc[sector, cluster_sector] == 1:
                    adj = True
            if adj==True:
                sector_list.append(sector)
                equip_to_append = equip
                break
        if equip_to_append is None:
            continue
        else:
            clustering_df_s.set_value(equip_to_append, "sector_list", sector_list)
            temp_dist_matrix.drop(sector, inplace=True)

solutions_list_s = []
for equip, sector_list in clustering_df_s.sector_list.iteritems():
    for sector in sector_list:
            solutions_list_s.append((sector, equip))

clustering_df_s.sample(5)

,Total,proporcao_fluxo,ideal_num_sectors,avg_distance,sector_list
Equipamento,,,,,
FS569JOI,470.5,0.026552,19,264.520003,"[420910205000007, 420910205000003]"
FS594JOI,62.0,0.003499,3,142.678411,"[420910205000449, 420910205000434, 42091020500..."
FS614JOI,127.0,0.007167,5,199.835360,"[420910205000344, 420910205000342, 42091020500..."
FS572JOI,110.0,0.006208,4,99.579647,"[420910205000597, 420910205000081, 42091020500..."
FS616JOI,125.5,0.007082,5,465.795966,"[420910205000143, 420910205000142, 42091020500..."


Clustering algorithm - Looping over radars

In [50]:
temp_dist_matrix = df_distance_matrix.copy()
clustering_df_e = flow_per_equip.copy()
clustering_df_e["avg_distance"] = 0
clustering_df_e["sector_list"] = [[] for _ in range(len(flow_per_equip))]


remaining = len(temp_dist_matrix) 
remaining_prior_loop = 0
while remaining>0:
    if remaining == remaining_prior_loop:
        print("No cluster found for", str(remaining), "sectors")
        print(temp_dist_matrix.index)
        break
    print(len(temp_dist_matrix))
    for equip, data in clustering_df_e.iterrows():
        adj = False
        i=0
        while adj == False:
            if i>=(len(temp_dist_matrix)):
                break
            closest_sector = temp_dist_matrix.loc[:,equip].sort_values().index[i]
            #Check adjacency
            sector_list = data["sector_list"]
            if not(bool(sector_list)):
                sector_list.append(closest_sector)
                clustering_df_e.set_value(equip, "sector_list", sector_list)
                temp_dist_matrix.drop(closest_sector, inplace=True)
                break
            for cluster_sector in sector_list:
                if adjacency_matrix.loc[closest_sector, cluster_sector] == 1:
                    adj = True
                    break    
            if adj==True:
                sector_list.append(closest_sector)
                clustering_df_e.set_value(equip, "sector_list", sector_list)
                temp_dist_matrix.drop(closest_sector, inplace=True)
            i += 1
    remaining_prior_loop = remaining
    remaining = len(temp_dist_matrix) 

solutions_list_e = []
for equip, sector_list in clustering_df_e.sector_list.iteritems():
    for sector in sector_list:
            solutions_list_e.append((sector, equip))

715
620
525
432
349
281
225
181
144
114
87
67
51
40
30
22
15
10
7
5
3
2
1


Optimization code:

In [ ]:
start = time.time()
prob = pulp.LpProblem("Designation Problem", pulp.LpMinimize)
possibleChoices = [(sector,equip) for sector in sector_list for equip in equip_list]

choice_equip_sector = pulp.LpVariable.dicts("choice",(sector_list,equip_list),0,1,pulp.LpInteger)

#objective function
objective_function = [choice_equip_sector[sector][equip]*df_distance_matrix.loc[sector,equip]
                      for (sector,equip) in possibleChoices]
prob += pulp.lpSum(objective_function)

#each sector must have at most 1 equipment
for sector in sector_list:
    vars_to_sum = [choice_equip_sector[sector][equip] for equip in equip_list]
    prob += pulp.lpSum(vars_to_sum) == 1
    
#each equipment must have at least 1 sector
for equip in equip_list:
    vars_to_sum = [choice_equip_sector[sector][equip] for sector in sector_list]
    prob += pulp.lpSum(vars_to_sum) >= 1

# #each equipment must have at most "num_sectors" sectors
# for equip in equip_list:
#     vars_to_sum = [choice_equip_sector[sector][equip] for sector in sector_list]
#     prob += pulp.lpSum(vars_to_sum) <= total_per_equip.loc[equip, "num_sectors"]
#     #prob += pulp.lpSum(vars_to_sum) <= 8

#only contiguous sectors are allowed

for j in range(0, len(sector_list)):
    for k in range(j, len(sector_list)):
        vars_to_sum = []
        for i in equip_list:
            sector1 = sector_list[j]
            sector2 = sector_list[k]
            vars_to_sum.append(choice_equip_sector[sector1][i])
            vars_to_sum.append(choice_equip_sector[sector2][i])
        prob += pulp.lpSum(vars_to_sum) <= (1 + adjacency_matrix.loc[sector1,sector2])

end = time.time()
elapsed_time = str(int(end-start))
print("Começando a resolução...", elapsed_time)
prob.solve(pulp.PULP_CBC_CMD(fracGap=0, msg=True))
print("Status: ", pulp.LpStatus[prob.status])
end = time.time()
elapsed_time = str(int(end-start))
print("Optimization run in " + elapsed_time + " seconds.")

solutions_list = []
for sector in sector_list:
    for equip in equip_list:
        if choice_equip_sector[sector][equip].value() == 1.0:
            solutions_list.append((sector, equip))

In [52]:
#Solution - sweeping sectors
print("Solution - sweeping sectors")
df_solutions_s = pd.DataFrame(solutions_list_s, columns=["Sector", "Equipment"])
print("Number of sectors after optimization: ", df_solutions_s["Sector"].nunique())
print("Number of radars after optimization: ", df_solutions_s["Equipment"].nunique())
display(df_solutions_s.groupby("Equipment").agg("count").sort_values("Sector", ascending=False).head(15))

#Solution - sweeping radars
print("Solution - sweeping radars")
df_solutions_e = pd.DataFrame(solutions_list_e, columns=["Sector", "Equipment"])
print("Number of sectors after optimization: ", df_solutions_e["Sector"].nunique())
print("Number of radars after optimization: ", df_solutions_e["Equipment"].nunique())
display(df_solutions_e.groupby("Equipment").agg("count").sort_values("Sector", ascending=False).head(15))

Solution - sweeping sectors
Number of sectors after optimization:  715
Number of radars after optimization:  95


,Sector
Equipment,
FS652JOI,49
FS563JOI,34
FS576JOI,24
FS625JOI,24
FS580JOI,23
FS598JOI,22
FS552JOI,20
FS601JOI,16
FS643JOI,15


Solution - sweeping radars
Number of sectors after optimization:  715
Number of radars after optimization:  95


,Sector
Equipment,
FS580JOI,23
FS552JOI,20
FS551JOI,18
FS652JOI,17
FS597JOI,17
FS598JOI,16
FS583JOI,16
FS614JOI,15
FS561JOI,14


In [53]:
#df_solutions_s.to_csv(project_dir + "/data/processed/od/sweeping_sectors.csv", index=False)
df_solutions_e.to_csv(project_dir + "/data/processed/od/sweeping_radars.csv", index=False)